In [72]:
import psycopg2
import pandas as pd
import configparser

# Read and import environement variables from cred.ini
config = configparser.ConfigParser()
config.sections()

config.read('cred.ini')

#connect to source database on the cloud using cred.ini source variables
db_source_conn = psycopg2.connect(
    database=config['SOURCE']['database'], 
    user=config['SOURCE']['user'], 
    password=config['SOURCE']['password'], 
    host=config['SOURCE']['host'], 
    port= config['SOURCE']['port']
    )

db_source_cursor = db_source_conn.cursor()

# Query to get all the data from the public.Hive table
db_source_cursor.execute(
    'select * from public."Hive"'
    )

# fetch all data 
result_source = db_source_cursor.fetchall()

In [75]:
df_hive = pd.DataFrame(result_source, columns=['id','createdAt','updatedAt','description','userId','name'])
df_hive.to_csv('./New Data/hive_data.csv',sep=';')
df_hive.tail()

,id,createdAt,updatedAt,description,userId,name
0,87067d94-dc50-4afd-8c0d-be41ce483092,2024-05-23 11:26:17.608,2024-05-23 11:26:17.608,my hive description,28b03d65-6603-4eb6-b67d-81bb721213e0,my hive
1,44b3fa36-a548-454a-8925-c46e08d159d9,2024-05-23 11:27:59.659,2024-05-23 11:27:59.659,my hive description,28b03d65-6603-4eb6-b67d-81bb721213e0,ruche2


In [86]:
db_source_conn.close()

In [94]:
db_target_conn = psycopg2.connect(
    database=config['TARGET']['database'], 
    user=config['TARGET']['user'], 
    password= "I'llbethere4U" ,
    host=config['TARGET']['host'], 
    port= config['TARGET']['port']
    )




db_target_cursor = db_target_conn.cursor()



drop_table_hive = """ 
    ALTER TABLE lake.hive_data DROP CONSTRAINT hive_data_hive_id_fkey; 
    DROP TABLE lake.hive 
"""

drop_table_user= """ 
    ALTER TABLE lake.hive_data DROP CONSTRAINT hive_data_hive_id_fkey;
    DROP TABLE lake.hive 
"""

drop_table_hive_data = """ 
    ALTER TABLE lake.hive_data DROP CONSTRAINT hive_data_hive_id_fkey; 
    DROP TABLE lake.hive 
"""

#drop_table_session = """ 
#     ALTER TABLE lake.hive_data DROP CONSTRAINT hive_data_hive_id_fkey; 
#     DROP TABLE lake.hive 
# """





create_table_hive = """
    CREATE TABLE lake.hive (
    id TEXT PRIMARY KEY NOT NULL,
    userid TEXT,
    name TEXT,
    createdat TIMESTAMP,
    updatedat TIMESTAMP,
    description TEXT,
    FOREIGN KEY (userid) REFERENCES users(id)
);
"""

db_target_cursor.execute(drop_table_hive)

db_target_cursor.execute(create_table_hive)

db_target_conn.commit()


result_target = db_target_cursor.fetchall()


UndefinedColumn: column "id" referenced in foreign key constraint does not exist


In [57]:
result_target

[]

In [85]:
with open('hive_data.csv', 'r') as f:
    next(f) # Skip the header row.
    db_target_cursor.copy_from(f, 'hive', sep=';')



InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
